# Digital Marketing Campaign Optimization

**Goal**: Maximize conversions by finding optimal intervention strategies

This notebook demonstrates using Intervention Search to optimize digital marketing campaigns by identifying which levers (budget, targeting, creative quality, etc.) to adjust for maximum conversions.

## 1. Load Marketing Campaign Data

In [ ]:
import pandas as pd
import numpy as np
import networkx as nx
import warnings
warnings.filterwarnings('ignore')

# Load data
df = pd.read_csv('data/marketing_data.csv')
print(f"Loaded {len(df)} marketing campaigns")
print(f"\nKey metrics:")
print(f"  • Avg conversions: {df['conversions'].mean():.1f}")
print(f"  • Avg CPA: ${df['cost_per_acquisition'].mean():.2f}")
print(f"  • Avg CTR: {df['click_through_rate'].mean():.2f}%")
df.head()

## 2. Define Marketing Causal Graph

**Causal Structure:**
- `ad_budget → impressions → clicks → conversions`
- `audience_size → impressions`
- `day_of_week → impressions`
- `targeting_quality → click_through_rate → clicks`
- `ad_creative_quality → click_through_rate`
- `landing_page_quality → conversion_rate → conversions`

In [ ]:
# Define nodes
nodes = ['ad_budget', 'targeting_quality', 'ad_creative_quality', 
         'landing_page_quality', 'audience_size', 'day_of_week',
         'impressions', 'click_through_rate', 'clicks', 
         'conversion_rate', 'conversions']

# Define causal edges
edges = [
    ('ad_budget', 'impressions'),
    ('audience_size', 'impressions'),
    ('day_of_week', 'impressions'),
    ('targeting_quality', 'click_through_rate'),
    ('ad_creative_quality', 'click_through_rate'),
    ('impressions', 'clicks'),
    ('click_through_rate', 'clicks'),
    ('landing_page_quality', 'conversion_rate'),
    ('clicks', 'conversions'),
    ('conversion_rate', 'conversions')
]

# Create adjacency matrix
adj_matrix = pd.DataFrame(0, index=nodes, columns=nodes)
for parent, child in edges:
    adj_matrix.loc[parent, child] = 1

print("Marketing Causal Graph Structure:")
print(f"  • Nodes: {len(nodes)}")
print(f"  • Edges: {len(edges)}")

## 3. Train Causal Models

In [ ]:
from ht_categ import HT, HTConfig

# Train HT model
config = HTConfig(graph=adj_matrix, model_type='XGBoost')
ht_model = HT(config)
ht_model.train(df)

print("✓ Causal models trained")
print(f"\nModel Performance (R²):")
for node, metrics in sorted(ht_model.model_metrics.items(), 
                            key=lambda x: x[1].get('r2', 0), reverse=True):
    if 'r2' in metrics:
        quality = '🟢' if metrics['r2'] > 0.7 else '🟡' if metrics['r2'] > 0.5 else '🔴'
        print(f"  {quality} {node}: {metrics['r2']:.3f}")

## 4. Scenario 1: Increase Conversions by 25%

In [ ]:
from intervention_search import InterventionSearch

# Initialize searcher
searcher = InterventionSearch(
    graph=ht_model.graph,
    ht_model=ht_model,
    n_simulations=1000
)

# Find interventions for +25% conversions
results_25 = searcher.find_interventions(
    target_outcome='conversions',
    target_change=25.0,
    tolerance=4.0,
    confidence_level=0.90,
    max_intervention_pct=30.0,
    verbose=True
)

In [ ]:
# Display best intervention
best = results_25['best_intervention']

print("\n" + "="*70)
print("BEST INTERVENTION: +25% Conversions")
print("="*70)
print(f"\n📊 Intervene on: {', '.join(best['nodes'])}")
print(f"\n🎯 Required Changes:")
for node, change in best['required_pct_changes'].items():
    baseline = ht_model.baseline_stats[node]['mean']
    new_value = baseline * (1 + change/100)
    print(f"  • {node}: {change:+.1f}%")
    print(f"    (from {baseline:.1f} to {new_value:.1f})")

print(f"\n📈 Expected Results:")
print(f"  • Predicted effect: {best['actual_effect']:+.1f}%")
print(f"  • 90% CI: [{best['ci_90'][0]:+.1f}%, {best['ci_90'][1]:+.1f}%]")
print(f"  • Confidence: {best['confidence']:.0%}")
print(f"  • Quality Grade: {best['quality']['overall_grade']}")
print("="*70)

## 5. Scenario 2: Multi-Node Intervention Strategy

Explore combinations of interventions for more robust improvements

In [ ]:
# Search with combinations allowed
results_combo = searcher.find_interventions(
    target_outcome='conversions',
    target_change=25.0,
    tolerance=4.0,
    confidence_level=0.90,
    allow_combinations=True,  # Enable 2-node combinations
    max_intervention_pct=20.0,  # Smaller changes per node
    verbose=True
)

# Show top combinations
print("\nTop 3 Intervention Combinations:\n")
for i, candidate in enumerate(results_combo['all_candidates'][:3], 1):
    print(f"{i}. Nodes: {', '.join(candidate['nodes'])}")
    print(f"   Effect: {candidate['actual_effect']:+.1f}% ± {candidate['uncertainty']:.1f}%")
    print(f"   Confidence: {candidate['confidence']:.0%}")
    for node, change in candidate['required_pct_changes'].items():
        print(f"     - {node}: {change:+.1f}%")
    print()

## 6. Comparative Analysis: Single vs Multi-Node

Compare different intervention strategies

In [ ]:
# Create comparison table
comparison = []
for i, (label, res) in enumerate([('Single Node', results_25), 
                                    ('Multi-Node', results_combo)], 1):
    best = res['best_intervention']
    comparison.append({
        'Strategy': label,
        'Nodes': ', '.join(best['nodes']),
        'Effect': f"{best['actual_effect']:+.1f}%",
        'CI Width': f"{best['ci_90'][1] - best['ci_90'][0]:.1f}%",
        'Confidence': f"{best['confidence']:.0%}",
        'Grade': best['quality']['overall_grade']
    })

comp_df = pd.DataFrame(comparison)
print("\nStrategy Comparison:")
print(comp_df.to_string(index=False))

## 7. Cost-Effectiveness Analysis

Estimate cost implications of different interventions

In [ ]:
# Estimate costs for different interventions
def estimate_intervention_cost(intervention, baseline_stats):
    """Rough cost estimation for marketing interventions"""
    cost_map = {
        'ad_budget': 1.0,  # Direct cost multiplier
        'targeting_quality': 0.3,  # Platform fees for better targeting
        'ad_creative_quality': 0.5,  # Creative production costs
        'landing_page_quality': 0.4,  # Development costs
    }
    
    total_cost = 0
    for node, pct_change in intervention['required_pct_changes'].items():
        if node in cost_map:
            baseline = baseline_stats[node]['mean']
            change_amount = baseline * abs(pct_change) / 100
            total_cost += change_amount * cost_map[node]
    
    return total_cost

# Analyze top 5 interventions by cost-effectiveness
print("\nCost-Effectiveness Analysis (Top 5):\n")
for i, candidate in enumerate(results_combo['all_candidates'][:5], 1):
    cost = estimate_intervention_cost(candidate, ht_model.baseline_stats)
    effect = candidate['actual_effect']
    roi = effect / (cost + 1) if cost > 0 else effect
    
    print(f"{i}. {', '.join(candidate['nodes'])}")
    print(f"   Est. Cost: ${cost:.0f} | Effect: {effect:+.1f}% | ROI: {roi:.2f}")
    print()

## 8. Key Insights & Recommendations

### 🎯 Insights:

1. **Primary Drivers**: The analysis reveals which marketing levers have the strongest causal impact on conversions
2. **Trade-offs**: Single-node interventions may require larger changes vs. multi-node strategies with smaller, distributed changes
3. **Confidence**: Higher quality grades indicate more reliable model predictions for those paths
4. **Cost-Effectiveness**: Not all interventions with similar effects have the same cost implications

### 💡 Recommendations:

1. **Start with High-ROI Interventions**: Prioritize changes that deliver strong effects at lower cost
2. **Monitor Model Quality**: Focus on intervention paths with R² > 0.7 for highest reliability
3. **A/B Test**: Validate predictions with controlled experiments before full rollout
4. **Iterative Approach**: Implement changes gradually and measure actual vs. predicted outcomes

### ⚠️ Caveats:

- Confidence intervals account for model uncertainty but not external factors
- Cost estimates are illustrative - actual costs may vary
- Results assume causal graph structure is correct

## Summary

This notebook demonstrated:
- ✅ Building causal models for digital marketing
- ✅ Finding optimal single and multi-node interventions
- ✅ Comparing intervention strategies
- ✅ Analyzing cost-effectiveness
- ✅ Generating actionable business recommendations

The Intervention Search system enables data-driven decision making by:
- Quantifying uncertainty through Monte Carlo simulation
- Validating model quality for each causal path
- Ranking interventions by multiple objectives (effect, confidence, simplicity)